In [1]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from MuonScaRe import pt_resol, pt_scale, pt_resol_var, pt_scale_var
import correctionlib

file_dict = {"root://eos.cms.rcac.purdue.edu:1094//store/data/Run2022C/Muon/NANOAOD/16Dec2023-v1/2550000/019b0054-1684-410d-9fb2-837f92da4956.root": {"object_path": "Events", "steps": [[0, 10], [10, 20]], "uuid": "b16b5dea-fbcd-11ed-bae7-a2a0b8bcbeef"}}



events_data = NanoEventsFactory.from_root(
    file_dict,
    schemaclass=NanoAODSchema,
    # steps_per_file = 2000,
    metadata={"dataset": "DYJets"},
    uproot_options={"timeout":2400},
).events()

# events_data = NanoEventsFactory.from_root(
#     'root://cms-xrd-global.cern.ch//store/data/Run2022C/Muon/NANOAOD/\
# 22Sep2023-v1/2520000/393889f7-173f-48ba-ad46-94828a4ee4b2.root',
#     schemaclass=NanoAODSchema,
# ).events()


file_dict = {"root://eos.cms.rcac.purdue.edu:1094//store/mc/Run3Summer22NanoAODv12/DYto2L-2Jets_MLL-50_0J_TuneCP5_13p6TeV_amcatnloFXFX-pythia8/NANOAODSIM/130X_mcRun3_2022_realistic_v5-v2/2520000/1e9fd5a7-16b9-4dfc-92a1-1b2b03631dde.root": {"object_path": "Events", "steps": [[0, 10], [10, 20]], "uuid": "b16b5dea-fbcd-11ed-bae7-a2a0b8bcbeef"}}

events_mc = NanoEventsFactory.from_root(
    file_dict,
    schemaclass=NanoAODSchema,
    # steps_per_file = 2000,
    metadata={"dataset": "DYJets"},
    uproot_options={"timeout":2400},
).events()

# events_mc = NanoEventsFactory.from_root(
#     'root://cms-xrd-global.cern.ch//store/mc/Run3Summer22NanoAODv12/\
# DYto2L-2Jets_MLL-50_TuneCP5_13p6TeV_amcatnloFXFX-pythia8/\
# NANOAODSIM/130X_mcRun3_2022_realistic_v5-v2/40000/4fa63a35-456c-4d69-9400-356033e505f1.root',
#     schemaclass=NanoAODSchema,
# ).events()

cset = correctionlib.CorrectionSet.from_file("../data/roch_corr/2022_Summer22.json")



/depot/cms/kernels/root632/lib/python3.12/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [2]:
# Data: only scale correction to gen Z peak
events_data["Muon", "ptcorr"] = pt_scale(
    1, # 1 for data, 0 for mc 
    events_data.Muon.pt, 
    events_data.Muon.eta, 
    events_data.Muon.phi, 
    events_data.Muon.charge, 
    cset, 
    nested=True # for awkward arrays. Set False for 1d arrays
)

There are dask.awkward<sum, type=Scalar, dtype=int64> events with muon pt outside of [26,200] GeV. Setting those entries to their initial value.
There are dask.awkward<add, type=Scalar, dtype=int64> nan entries in the corrected pt. This might be due to the number of tracker layers hitting boundaries. Setting those entries to their initial value.


In [3]:
events_data["Muon", "ptcorr"].compute()

<Array [[12.3, 5.76, 4.3], [38.2], ..., [31.5]] type='20 * var * float64'>

/depot/cms/kernels/root632/lib/python3.12/site-packages/awkward/_nplikes/array_module.py:251: RuntimeWarning: invalid value encountered in power
  return impl(*broadcasted_args, **(kwargs or {}))


In [4]:

# MC: both scale correction to gen Z peak AND resolution correction to Z width in data
events_mc["Muon", "ptscalecorr"] = pt_scale(
    0, 
    events_mc.Muon.pt, 
    events_mc.Muon.eta, 
    events_mc.Muon.phi, 
    events_mc.Muon.charge, 
    cset, 
    nested=True
)

events_mc["Muon", "ptcorr"] = pt_resol(
    events_mc.Muon.ptscalecorr, 
    events_mc.Muon.eta, 
    events_mc.Muon.nTrackerLayers, 
    cset, 
    nested=True
)

# # uncertainties
# events_mc["Muon", "ptscalecorr_up"] = pt_scale_var(
#     events_mc.Muon.ptcorr, 
#     events_mc.Muon.eta, 
#     events_mc.Muon.phi, 
#     events_mc.Muon.charge,
#     "up",
#     cset, 
#     nested=True
# )
# events_mc["Muon", "ptscalecorr_dn"] = pt_scale_var(
#     events_mc.Muon.ptcorr, 
#     events_mc.Muon.eta, 
#     events_mc.Muon.phi, 
#     events_mc.Muon.charge,
#     "dn",
#     cset, 
#     nested=True
# )

# events_mc["Muon", "ptcorr_resolup"] = pt_resol_var(
#     events_mc.Muon.ptscalecorr, 
#     events_mc.Muon.ptcorr, 
#     events_mc.Muon.eta, 
#     "up",
#     cset, 
#     nested=True
# )
# events_mc["Muon", "ptcorr_resoldn"] = pt_resol_var(
#     events_mc.Muon.ptscalecorr, 
#     events_mc.Muon.ptcorr, 
#     events_mc.Muon.eta, 
#     "dn",
#     cset, 
#     nested=True
# )

There are dask.awkward<sum, type=Scalar, dtype=int64> events with muon pt outside of [26,200] GeV. Setting those entries to their initial value.
There are dask.awkward<add, type=Scalar, dtype=int64> nan entries in the corrected pt. This might be due to the number of tracker layers hitting boundaries. Setting those entries to their initial value.
rand_vals: [[], [0.229], [0.447], [], [], [], ..., [], [], [0.709], [], [0.526, 0.145]]
nmuons: [0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 2, 0, 2, 0, 1, 0, 0, 1, 0, 2]
There are dask.awkward<sum, type=Scalar, dtype=int64> events with muon pt outside of [26,200] GeV. Setting those entries to their initial value.
There are dask.awkward<add, type=Scalar, dtype=int64> nan entries in the corrected pt. This might be due to the number of tracker layers hitting boundaries. Setting those entries to their initial value.


In [5]:
print(events_mc["Muon", "ptscalecorr"].compute())
print(events_mc["Muon", "ptcorr"].compute())

[[], [3.9], [38.8], [], [], [], [], ..., [9.04], [], [], [3.4], [], [51.6, 36]]
[[], [3.9], [38.8], [], [], [], ..., [9.04], [], [], [3.4], [], [51.6, 35.7]]


In [9]:
# uncertainties
events_mc["Muon", "ptscalecorr_up"] = pt_scale_var(
    events_mc.Muon.ptcorr, 
    events_mc.Muon.eta, 
    events_mc.Muon.phi, 
    events_mc.Muon.charge,
    "up",
    cset, 
    nested=True
)
events_mc["Muon", "ptscalecorr_dn"] = pt_scale_var(
    events_mc.Muon.ptcorr, 
    events_mc.Muon.eta, 
    events_mc.Muon.phi, 
    events_mc.Muon.charge,
    "dn",
    cset, 
    nested=True
)

events_mc["Muon", "ptcorr_resolup"] = pt_resol_var(
    events_mc.Muon.ptscalecorr, 
    events_mc.Muon.ptcorr, 
    events_mc.Muon.eta, 
    "up",
    cset, 
    nested=True
)
events_mc["Muon", "ptcorr_resoldn"] = pt_resol_var(
    events_mc.Muon.ptscalecorr, 
    events_mc.Muon.ptcorr, 
    events_mc.Muon.eta, 
    "dn",
    cset, 
    nested=True
)

In [10]:
print(events_mc["Muon", "ptscalecorr_up"].compute())
print(events_mc["Muon", "ptscalecorr_dn"].compute())
print(events_mc["Muon", "ptcorr_resolup"].compute())
print(events_mc["Muon", "ptcorr_resoldn"].compute())

[[], [3.9], [38.8], [], [], [], ..., [9.04], [], [], [3.4], [], [51.6, 35.7]]
[[], [3.9], [38.7], [], [], [], ..., [9.04], [], [], [3.4], [], [51.6, 35.6]]
[[], [3.9], [38.8], [], [], [], ..., [9.04], [], [], [3.4], [], [51.6, 35.6]]
[[], [3.9], [38.8], [], [], [], ..., [9.04], [], [], [3.4], [], [51.6, 35.7]]


In [6]:
import awkward as ak
A = ak.Array([[None, 1], [1,2] , [4]])
# ak.is_none(A)
import numpy as np
np.isnan(A)

<Array [[None, False], [False, ...], [False]] type='3 * var * ?bool'>

In [13]:
is_mc = False
data_idx = 0 if is_mc else 1
data_idx

1